Full name: Tô Hiển Vinh

Student ID: 21120365

# HW4: Parallel Radix Sort

**To compile your file, you can use this command:** \
`nvcc tên-file.cu -o tên-file-chạy` \
***You can use Vietnamese to anwser the questions***

In [7]:
!rm -rf '-CSC14120-Lab-04'

rm: invalid option -- 'C'
Try 'rm ./-CSC14120-Lab-04' to remove the file '-CSC14120-Lab-04'.
Try 'rm --help' for more information.


In [8]:
!if ! [ -f "HW4.cu" ]; then \
    echo "Missing source code. Attempt to clone from Github..."; \
    git clone "https://github.com/ezio-noir/-CSC14120-Lab-04.git" "src"; \
    cp "src/HW4.cu" .; \
else \
    echo "Found source code."; \
fi

Missing source code. Attempt to clone from Github...
Cloning into 'src'...
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 5 (delta 0), reused 5 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (5/5), done.


In [9]:
!nvcc HW4.cu -o HW4

In [10]:
# Run with block size 256
!./HW4 256

**********GPU info**********
Name: Tesla T4
Compute capability: 7.5
Num SMs: 40
Max num threads per SM: 1024
Max num warps per SM: 32
GMEM: 15835660288 byte
SMEM per SM: 65536 byte
SMEM per block: 49152 byte
****************************

Input size: 16777217

Radix Sort by host
Time: 9497.573 ms

Radix Sort by device
Time: 1025.334 ms
CORRECT :)


In [12]:
# Run with block size 512
!./HW4 512

**********GPU info**********
Name: Tesla T4
Compute capability: 7.5
Num SMs: 40
Max num threads per SM: 1024
Max num warps per SM: 32
GMEM: 15835660288 byte
SMEM per SM: 65536 byte
SMEM per block: 49152 byte
****************************

Input size: 16777217

Radix Sort by host
Time: 10125.523 ms

Radix Sort by device
Time: 579.654 ms
CORRECT :)


In [13]:
# Run with block size 1024
!./HW4 1024

**********GPU info**********
Name: Tesla T4
Compute capability: 7.5
Num SMs: 40
Max num threads per SM: 1024
Max num warps per SM: 32
GMEM: 15835660288 byte
SMEM per SM: 65536 byte
SMEM per block: 49152 byte
****************************

Input size: 16777217

Radix Sort by host
Time: 9409.847 ms

Radix Sort by device
Time: 421.923 ms
CORRECT :)


- Device execution time seems to be reduced as the block size increases. Specifically in my runtime on Colab's Tesla T4, the execution time is as follows:  

|Version | Time (ms)|   
|------- | ---------|
| Host (run with block size = 256 version) | 9497.573 |
| Host (run with block size = 512 version) | 10125.523 |
| Host (run with block size = 1024 version) | 9409.847 |
| Device (block size = 256) | 1025.334 |
| Device (block size = 512) | 579.654 |
| Device (block size = 1024) | 421.923 |

- Kernels execution time (link to `nsys` reports: [Google Drive](https://drive.google.com/drive/folders/1zviQ9bWn3vwCjZ4rxAnvpCT2BB0BlkS7?usp=sharing)):

| Kernel | Average time (ms)|   
|------- | ---------|
| `g_extractBits` (block size = 256) | 0.520 - 0.559 |
| `g_extractBits` (block size = 512) | 0.524 - 0.589 |
| `g_extractBits` (block size = 1024) | 0.629 - 0.671 |
| `g_scan` (block size = 256) | 16.4 - 21.4 |
| `g_scan` (block size = 512) | 8.7 - 11.2 |
| `g_scan` (block size = 1024) | 4.9 - 5.3 |
| `g_computeRankAndAssign` (block size = 256) | 0.990 - 1.002 |
| `g_computeRankAndAssign` (block size = 512) | 0.991 - 1.002 |
| `g_computeRankAndAssign` (block size = 1024) | 0.995 - 0.998 |

- We can see that `g_scan` takes most of the device execution time among the kernels, and mainly contributes the performances differences between the versions. We can also notice that `g_scan`'s execution time and block size are inversely proportional (i.e. execution time halves when the block size doubles).

- Let $B = $ block size, $n = $ number of input elements (for simplicity, assume $n$ is an exponent of $2$), then $n_{B} = $ number of blocks $ = ⌈\frac{n}{2B}⌉$, since each block handles a section of $2B$ elements.
    - For each block $i$, the execution is made up of these steps:  
        1. Get block ID and load data from global memory into shared memory.
        2. Reduction phase.
        3. Post-reduction phase.
        4. Write block result to `block_sums` and wait for previous blocks to complete.
        5. Write final result to `out`.
    - Each block can execute step 1. and 5. concurrently and with same time (assumming getting block ID is neglible), since each has to perform loading $2B$ elements. Let $T_{1_i} \approx T_1$ is time cost of this step.
    - The reduction phase performs tree reduction on $2B$ elements, which make time complexity be $T_{2_i} \approx T_2 = \log_2{2B}$.
    - The post-reduction phase time complexity is $T_{3_i} \approx T_3 =
    \log_2{2B} - 1 = \log_2{B}$ (1 step less than reduction phase).
    - We can see that step 1, 2, and 3 are executed independenly by blocks, and cost roughly the same time.
    - In step 4, a block $i$ must wait for all block $j, j < i$  to complete their steps 4 (only then the `complete_block_count` increases to be equal to `block_id`).
    - So the total execution time for block $i$ is: $T_i = T_{1_i} + T_{2_i} + T_{3_i} + T_{4_i} + T_{5_i} ≈ T_1 + T_2 + T_3 + T_{4_i} + T_5 = T_1 + T_2 + T_3 + ∑_{j = 0}^{j = i - 1} T_4j + T_5 $.
    - Then the execution time of the last block is the longest: $T_1 + T_2 + T_3 + ∑_{j = 0}^{j = n_b - 1} T_4j + T_5$.
    - Since the kernel time is equal to the longest execution time of its blocks, we can see that the blocking of step 4 contributes to the difference in execution time when block size varies. For example, when the block size $B$ is doubled, then $n_b$ is halved, which make the last block wait less blocks before it to complete.